In [1]:
# -*- coding: utf-8 -*-
'''
@time: 2019/01/11 11:28
spytensor
'''

import os
import json
import numpy as np
import pandas as pd
import glob
import cv2
import os
import shutil
from IPython import embed
from sklearn.model_selection import train_test_split
np.random.seed(41)

#0为背景
classname_to_id = {'长马甲': 1, '古风': 2, '短马甲': 3, '背心上衣': 4, '背带裤': 5, '连体衣': 6, '吊带上衣': 7, '中裤': 8, '短袖衬衫': 9, '无袖上衣': 10,
                 '长袖衬衫': 11, '中等半身裙': 12, '长半身裙': 13, '长外套': 14, '短裙': 15, '无袖连衣裙': 16, '短裤': 17, '短外套': 18,
                 '长袖连衣裙': 19, '长袖上衣': 20, '长裤': 21, '短袖连衣裙': 22, '短袖上衣': 23, '古装': 2}

class Csv2CoCo:

    def __init__(self,image_dir,total_annos):
        self.images = []
        self.annotations = []
        self.categories = []
        self.img_id = 0
        self.ann_id = 0
        self.image_dir = image_dir
        self.total_annos = total_annos

    def save_coco_json(self, instance, save_path):
        json.dump(instance, open(save_path, 'w'), ensure_ascii=False, indent=2)  # indent=2 更加美观显示

    # 由txt文件构建COCO
    def to_coco(self, keys):
        self._init_categories()
        for key in keys:
            self.images.append(self._image(key))
            shapes = self.total_annos[key]
            for shape in shapes:
                bboxi = []
                for cor in shape[:-1]:
                    bboxi.append(int(cor))
                label = shape[-1]
                annotation = self._annotation(bboxi,label)
                self.annotations.append(annotation)
                self.ann_id += 1
            self.img_id += 1
        instance = {}
        instance['info'] = 'spytensor created'
        instance['license'] = ['license']
        instance['images'] = self.images
        instance['annotations'] = self.annotations
        instance['categories'] = self.categories
        return instance

    # 构建类别
    def _init_categories(self):
        for k, v in classname_to_id.items():
            category = {}
            category['id'] = v
            if k=='古装':
                continue
            category['name'] = k
            self.categories.append(category)

    # 构建COCO的image字段
    def _image(self, path):
        image = {}
#         print(path)
        img = cv2.imread(self.image_dir + path)
        image['height'] = img.shape[0]
        image['width'] = img.shape[1]
        image['id'] = self.img_id
        image['file_name'] = path.split(os.sep)[-2]+'_'+path.split(os.sep)[-1]
        return image

    # 构建COCO的annotation字段
    def _annotation(self, shape,label):
        # label = shape[-1]
        points = shape[:4]
        annotation = {}
        annotation['id'] = self.ann_id
        annotation['image_id'] = self.img_id
        annotation['category_id'] = int(classname_to_id[label])
        annotation['segmentation'] = self._get_seg(points)
        annotation['bbox'] = self._get_box(points)
        annotation['iscrowd'] = 0
        annotation['area'] = self._get_area(points)
        return annotation

    # COCO的格式： [x1,y1,w,h] 对应COCO的bbox格式
    def _get_box(self, points):
        min_x = points[0]
        min_y = points[1]
        max_x = points[2]
        max_y = points[3]
        return [min_x, min_y, max_x - min_x, max_y - min_y]
    # 计算面积
    def _get_area(self, points):
        min_x = points[0]
        min_y = points[1]
        max_x = points[2]
        max_y = points[3]
        return (max_x - min_x+1) * (max_y - min_y+1)
    # segmentation
    def _get_seg(self, points):
        min_x = points[0]
        min_y = points[1]
        max_x = points[2]
        max_y = points[3]
        h = max_y - min_y
        w = max_x - min_x
        a = []
        a.append([min_x,min_y, min_x,min_y+0.5*h, min_x,max_y, min_x+0.5*w,max_y, max_x,max_y, max_x,max_y-0.5*h, max_x,min_y, max_x-0.5*w,min_y])
        return a
   


   

In [5]:
def creat_csv_image_dataset(data_root_path='origin_data/',mode='train'):
    dataset_paths = glob.glob(data_root_path+mode+'*')
    # 图像库中标注
    img_ann_folder_paths = []  # 所有data/train_dataset_part<n>/image_annotatonl中所有文件夹

#     # 视频库中标注
#     video_ann_paths = []  # 所有data/train_dataset_part<n>/video_annotation中所有json文件


    for dataset_path in dataset_paths:
        img_ann_folder_paths.extend(glob.glob(dataset_path + '/image_annotation/*'))

#         video_ann_paths.extend(glob.glob(dataset_path + '/video_annotation/*.json'))

    image_db = []
    for img_ann_folder_path in img_ann_folder_paths[:]:
        split_list = img_ann_folder_path.split('/')
        img_folder_path = data_root_path + split_list[1] + '/image/' + split_list[-1] + '/'
        json_paths = glob.glob(img_ann_folder_path + '/*.json')
        for json_path in json_paths:
            with open(json_path, 'r') as f:
                img_anns = json.load(f)
            if len(img_anns['annotations']) > 0:
                img_path = img_folder_path + json_path.split('/')[-1].split('.')[0] + '.jpg'
                for ann in img_anns['annotations']:
                    box = ann['box']
                    label = ann['label']
                    image_db.append([img_path, box[0],box[1],box[2],box[3],label])
                
    image_db = pd.DataFrame(image_db)

    image_db.to_csv(data_root_path+mode+'_image_dataset.csv', index=False)
    print('已生成csv路径文件：' + data_root_path+mode+'_image_dataset.csv')
    print(image_db.info())

In [9]:
creat_csv_image_dataset()

已生成csv路径文件：origin_data/test_image_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63306 entries, 0 to 63305
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       63306 non-null  object
 1   1       63306 non-null  int64 
 2   2       63306 non-null  int64 
 3   3       63306 non-null  int64 
 4   4       63306 non-null  int64 
 5   5       63306 non-null  object
dtypes: int64(4), object(2)
memory usage: 2.9+ MB
None


In [4]:
csv_file = "origin_data/train_image_dataset.csv"
image_dir = ""
saved_coco_path = ""
# 整合csv格式标注文件
total_csv_annotations = {}
annotations = pd.read_csv(csv_file).values
# num=0
for annotation in annotations: # origin_data/validation_dataset_part1/image/064598/0.jpg,233,156,530,755,短袖连衣裙
#     key = annotation[0].split(os.sep)[-2]+'_'+annotation[0].split(os.sep)[-1]
    key = annotation[0]
    value = np.array([annotation[1:]])
    if key in total_csv_annotations.keys():
        total_csv_annotations[key] = np.concatenate((total_csv_annotations[key],value),axis=0)
    else:
        total_csv_annotations[key] = value
#     num+=1
#     if num==10:
#         break
# 按照键值划分数据
total_keys = list(total_csv_annotations.keys())
train_keys, val_keys = train_test_split(total_keys, test_size=0.2)
print("train_n:", len(train_keys), 'val_n:', len(val_keys))
# 创建必须的文件夹
if not os.path.exists('%scoco/annotations/'%saved_coco_path):
    os.makedirs('%scoco/annotations/'%saved_coco_path)
if not os.path.exists('%scoco/images/train2017/'%saved_coco_path):
    os.makedirs('%scoco/images/train2017/'%saved_coco_path)
if not os.path.exists('%scoco/images/val2017/'%saved_coco_path):
    os.makedirs('%scoco/images/val2017/'%saved_coco_path)
# 把训练集转化为COCO的json格式
l2c_train = Csv2CoCo(image_dir=image_dir,total_annos=total_csv_annotations)
train_instance = l2c_train.to_coco(train_keys)
l2c_train.save_coco_json(train_instance, '%scoco/annotations/instances_train2017.json'%saved_coco_path)
for file in train_keys:
    if not os.path.exists("%scoco/images/train2017/%s"%(saved_coco_path,file.split(os.sep)[-2]+'_'+file.split(os.sep)[-1])):
        print("%scoco/images/train2017/%s"%(saved_coco_path,file.split(os.sep)[-2]+'_'+file.split(os.sep)[-1]))
        shutil.copy(image_dir+file,"%scoco/images/train2017/%s"%(saved_coco_path,file.split(os.sep)[-2]+'_'+file.split(os.sep)[-1]))
for file in val_keys:
    if not os.path.exists("%scoco/images/val2017/%s"%(saved_coco_path,file.split(os.sep)[-2]+'_'+file.split(os.sep)[-1])):
        shutil.copy(image_dir+file,"%scoco/images/val2017/%s"%(saved_coco_path,file.split(os.sep)[-2]+'_'+file.split(os.sep)[-1]))
# 把验证集转化为COCO的json格式
l2c_val = Csv2CoCo(image_dir=image_dir,total_annos=total_csv_annotations)
val_instance = l2c_val.to_coco(val_keys)
l2c_val.save_coco_json(val_instance, '%scoco/annotations/instances_val2017.json'%saved_coco_path)

train_n: 120194 val_n: 30049
coco/images/train2017/002435_5.jpg
coco/images/train2017/097102_0.jpg
coco/images/train2017/104415_4.jpg
coco/images/train2017/077733_5.jpg
coco/images/train2017/097628_0.jpg
coco/images/train2017/070179_0.jpg
coco/images/train2017/065590_0.jpg
coco/images/train2017/110140_3.jpg
coco/images/train2017/007914_4.jpg
coco/images/train2017/056729_5.jpg
coco/images/train2017/029972_5.jpg
coco/images/train2017/044669_5.jpg
coco/images/train2017/019700_2.jpg
coco/images/train2017/042393_4.jpg
coco/images/train2017/027838_2.jpg
coco/images/train2017/113548_1.jpg
coco/images/train2017/097710_0.jpg
coco/images/train2017/005411_1.jpg
coco/images/train2017/025233_4.jpg
coco/images/train2017/019654_5.jpg
coco/images/train2017/091122_3.jpg
coco/images/train2017/044027_0.jpg
coco/images/train2017/020159_4.jpg
coco/images/train2017/076845_4.jpg
coco/images/train2017/112909_3.jpg
coco/images/train2017/018092_2.jpg
coco/images/train2017/025502_4.jpg
coco/images/train2017/0817

coco/images/train2017/004075_3.jpg
coco/images/train2017/068587_0.jpg
coco/images/train2017/021164_2.jpg
coco/images/train2017/040723_3.jpg
coco/images/train2017/009766_5.jpg
coco/images/train2017/074959_2.jpg
coco/images/train2017/056432_4.jpg
coco/images/train2017/017069_2.jpg
coco/images/train2017/064403_1.jpg
coco/images/train2017/097889_1.jpg
coco/images/train2017/090120_0.jpg
coco/images/train2017/063373_4.jpg
coco/images/train2017/028274_2.jpg
coco/images/train2017/064719_3.jpg
coco/images/train2017/105126_4.jpg
coco/images/train2017/034237_2.jpg
coco/images/train2017/018467_1.jpg
coco/images/train2017/066916_3.jpg
coco/images/train2017/116031_2.jpg
coco/images/train2017/052611_2.jpg
coco/images/train2017/028574_0.jpg
coco/images/train2017/004382_4.jpg
coco/images/train2017/005543_3.jpg
coco/images/train2017/030554_2.jpg
coco/images/train2017/024410_5.jpg
coco/images/train2017/072917_1.jpg
coco/images/train2017/081776_1.jpg
coco/images/train2017/052883_3.jpg
coco/images/train201

coco/images/train2017/009687_1.jpg
coco/images/train2017/010076_3.jpg
coco/images/train2017/056300_3.jpg
coco/images/train2017/025741_0.jpg
coco/images/train2017/088851_4.jpg
coco/images/train2017/003714_2.jpg
coco/images/train2017/042258_0.jpg
coco/images/train2017/001142_0.jpg
coco/images/train2017/011916_5.jpg
coco/images/train2017/036592_2.jpg
coco/images/train2017/061029_0.jpg
coco/images/train2017/060686_3.jpg
coco/images/train2017/098842_2.jpg
coco/images/train2017/043336_1.jpg
coco/images/train2017/043692_4.jpg
coco/images/train2017/041436_3.jpg
coco/images/train2017/062526_5.jpg
coco/images/train2017/044690_3.jpg
coco/images/train2017/015314_4.jpg
coco/images/train2017/026697_3.jpg
coco/images/train2017/028073_4.jpg
coco/images/train2017/047216_4.jpg
coco/images/train2017/017093_4.jpg
coco/images/train2017/019033_5.jpg
coco/images/train2017/118027_4.jpg
coco/images/train2017/045677_3.jpg
coco/images/train2017/104410_1.jpg
coco/images/train2017/113555_4.jpg
coco/images/train201

coco/images/train2017/010575_2.jpg
coco/images/train2017/042840_5.jpg
coco/images/train2017/114028_0.jpg
coco/images/train2017/007026_0.jpg
coco/images/train2017/025005_2.jpg
coco/images/train2017/017991_4.jpg
coco/images/train2017/055479_4.jpg
coco/images/train2017/034695_3.jpg
coco/images/train2017/003873_2.jpg
coco/images/train2017/058260_1.jpg
coco/images/train2017/023908_0.jpg
coco/images/train2017/004330_0.jpg
coco/images/train2017/114268_2.jpg
coco/images/train2017/053212_4.jpg
coco/images/train2017/111231_1.jpg
coco/images/train2017/017112_1.jpg
coco/images/train2017/026566_4.jpg
coco/images/train2017/011964_5.jpg
coco/images/train2017/013518_1.jpg
coco/images/train2017/010849_2.jpg
coco/images/train2017/017873_5.jpg
coco/images/train2017/053374_4.jpg
coco/images/train2017/006610_1.jpg
coco/images/train2017/012696_0.jpg
coco/images/train2017/086435_1.jpg
coco/images/train2017/029125_0.jpg
coco/images/train2017/017074_1.jpg
coco/images/train2017/015657_4.jpg
coco/images/train201

coco/images/train2017/064988_2.jpg
coco/images/train2017/049940_4.jpg
coco/images/train2017/043529_1.jpg
coco/images/train2017/075686_5.jpg
coco/images/train2017/069687_2.jpg
coco/images/train2017/037513_4.jpg
coco/images/train2017/009393_5.jpg
coco/images/train2017/010705_5.jpg
coco/images/train2017/085732_0.jpg
coco/images/train2017/038970_2.jpg
coco/images/train2017/079460_5.jpg
coco/images/train2017/000808_4.jpg
coco/images/train2017/049241_5.jpg
coco/images/train2017/000256_5.jpg
coco/images/train2017/116060_2.jpg
coco/images/train2017/057522_2.jpg
coco/images/train2017/030139_5.jpg
coco/images/train2017/073630_0.jpg
coco/images/train2017/063311_1.jpg
coco/images/train2017/021423_5.jpg
coco/images/train2017/103387_2.jpg
coco/images/train2017/029487_2.jpg
coco/images/train2017/119010_1.jpg
coco/images/train2017/017227_0.jpg
coco/images/train2017/018038_3.jpg
coco/images/train2017/034806_3.jpg
coco/images/train2017/064626_3.jpg
coco/images/train2017/090320_3.jpg
coco/images/train201

...

coco/images/train2017/011888_5.jpg
coco/images/train2017/030940_1.jpg
coco/images/train2017/049403_3.jpg
coco/images/train2017/027273_5.jpg
coco/images/train2017/051204_1.jpg
coco/images/train2017/031158_1.jpg
coco/images/train2017/031481_0.jpg
coco/images/train2017/049668_5.jpg
coco/images/train2017/031847_1.jpg
coco/images/train2017/082701_0.jpg
coco/images/train2017/067942_5.jpg
coco/images/train2017/065838_2.jpg
coco/images/train2017/034318_1.jpg
coco/images/train2017/017069_5.jpg
coco/images/train2017/031488_3.jpg
coco/images/train2017/017895_1.jpg
coco/images/train2017/070165_2.jpg
coco/images/train2017/005356_4.jpg
coco/images/train2017/048244_0.jpg
coco/images/train2017/058782_4.jpg
coco/images/train2017/035196_1.jpg
coco/images/train2017/042100_2.jpg
coco/images/train2017/111218_4.jpg
coco/images/train2017/003671_2.jpg
coco/images/train2017/018337_0.jpg
coco/images/train2017/074750_1.jpg
coco/images/train2017/110725_0.jpg
coco/images/train2017/080313_4.jpg
coco/images/train201

coco/images/train2017/019675_0.jpg
coco/images/train2017/069389_0.jpg
coco/images/train2017/060533_3.jpg
coco/images/train2017/011466_4.jpg
coco/images/train2017/018431_5.jpg
coco/images/train2017/105695_0.jpg
coco/images/train2017/016793_5.jpg
coco/images/train2017/032386_4.jpg
coco/images/train2017/081914_1.jpg
coco/images/train2017/080107_1.jpg
coco/images/train2017/088031_2.jpg
coco/images/train2017/010824_4.jpg
coco/images/train2017/078019_1.jpg
coco/images/train2017/093839_4.jpg
coco/images/train2017/017395_1.jpg
coco/images/train2017/022851_2.jpg
coco/images/train2017/050363_3.jpg
coco/images/train2017/081278_4.jpg
coco/images/train2017/022584_0.jpg
coco/images/train2017/026019_3.jpg
coco/images/train2017/105961_4.jpg
coco/images/train2017/034803_2.jpg
coco/images/train2017/117142_0.jpg
coco/images/train2017/017203_3.jpg
coco/images/train2017/006626_2.jpg
coco/images/train2017/030068_5.jpg
coco/images/train2017/034282_0.jpg
coco/images/train2017/105303_1.jpg
coco/images/train201

coco/images/train2017/030627_2.jpg
coco/images/train2017/022584_3.jpg
coco/images/train2017/072320_3.jpg
coco/images/train2017/020604_0.jpg
coco/images/train2017/069711_0.jpg
coco/images/train2017/112511_2.jpg
coco/images/train2017/055302_2.jpg
coco/images/train2017/095345_1.jpg
coco/images/train2017/076384_2.jpg
coco/images/train2017/035060_5.jpg
coco/images/train2017/062213_5.jpg
coco/images/train2017/096587_2.jpg
coco/images/train2017/077982_0.jpg
coco/images/train2017/073771_0.jpg
coco/images/train2017/115650_2.jpg
coco/images/train2017/024014_0.jpg
coco/images/train2017/027529_5.jpg
coco/images/train2017/039717_1.jpg
coco/images/train2017/027084_1.jpg
coco/images/train2017/029067_5.jpg
coco/images/train2017/000275_0.jpg
coco/images/train2017/025278_0.jpg
coco/images/train2017/115068_0.jpg
coco/images/train2017/032885_3.jpg
coco/images/train2017/000927_5.jpg
coco/images/train2017/058006_4.jpg
coco/images/train2017/023658_1.jpg
coco/images/train2017/048435_4.jpg
coco/images/train201

coco/images/train2017/076210_1.jpg
coco/images/train2017/041402_1.jpg
coco/images/train2017/064912_0.jpg
coco/images/train2017/057316_4.jpg
coco/images/train2017/016505_5.jpg
coco/images/train2017/005021_5.jpg
coco/images/train2017/058625_2.jpg
coco/images/train2017/038284_5.jpg
coco/images/train2017/005571_0.jpg
coco/images/train2017/005203_3.jpg
coco/images/train2017/056128_4.jpg
coco/images/train2017/026584_5.jpg
coco/images/train2017/006324_0.jpg
coco/images/train2017/093955_1.jpg
coco/images/train2017/025558_1.jpg
coco/images/train2017/063368_4.jpg
coco/images/train2017/072279_4.jpg
coco/images/train2017/002922_0.jpg
coco/images/train2017/002572_3.jpg
coco/images/train2017/034226_2.jpg
coco/images/train2017/069935_2.jpg
coco/images/train2017/050433_0.jpg
coco/images/train2017/039019_1.jpg
coco/images/train2017/001778_4.jpg
coco/images/train2017/069954_3.jpg
coco/images/train2017/070425_2.jpg
coco/images/train2017/029706_4.jpg
coco/images/train2017/069680_5.jpg
coco/images/train201

coco/images/train2017/015946_3.jpg
coco/images/train2017/104313_0.jpg
coco/images/train2017/047926_1.jpg
coco/images/train2017/060498_4.jpg
coco/images/train2017/022236_2.jpg
coco/images/train2017/070122_0.jpg
coco/images/train2017/050426_1.jpg
coco/images/train2017/045185_3.jpg
coco/images/train2017/087224_2.jpg
coco/images/train2017/017749_1.jpg
coco/images/train2017/030765_4.jpg
coco/images/train2017/043475_1.jpg
coco/images/train2017/003127_0.jpg
coco/images/train2017/067034_2.jpg
coco/images/train2017/080006_3.jpg
coco/images/train2017/066287_2.jpg
coco/images/train2017/002764_1.jpg
coco/images/train2017/053612_1.jpg
coco/images/train2017/032684_5.jpg
coco/images/train2017/054709_2.jpg
coco/images/train2017/087666_0.jpg
coco/images/train2017/004192_4.jpg
coco/images/train2017/092799_0.jpg
coco/images/train2017/019881_3.jpg
coco/images/train2017/017120_0.jpg
coco/images/train2017/011975_0.jpg
coco/images/train2017/010031_3.jpg
coco/images/train2017/051907_4.jpg
coco/images/train201

In [5]:
csv_file = "origin_data/test_image_dataset.csv"
image_dir = ""
saved_coco_path = ""
# 整合csv格式标注文件
total_csv_annotations = {}
annotations = pd.read_csv(csv_file).values
# num=0
for annotation in annotations: # origin_data/validation_dataset_part1/image/064598/0.jpg,233,156,530,755,短袖连衣裙
#     key = annotation[0].split(os.sep)[-2]+'_'+annotation[0].split(os.sep)[-1]
    key = annotation[0]
    value = np.array([annotation[1:]])
    if key in total_csv_annotations.keys():
        total_csv_annotations[key] = np.concatenate((total_csv_annotations[key],value),axis=0)
    else:
        total_csv_annotations[key] = value
#     num+=1
#     if num==10:
#         break
# 按照键值划分数据
test_keys = list(total_csv_annotations.keys())
print("test_n:", len(test_keys))
# 创建必须的文件夹
if not os.path.exists('%scoco/annotations/'%saved_coco_path):
    os.makedirs('%scoco/annotations/'%saved_coco_path)
if not os.path.exists('%scoco/images/train2017/'%saved_coco_path):
    os.makedirs('%scoco/images/train2017/'%saved_coco_path)
if not os.path.exists('%scoco/images/val2017/'%saved_coco_path):
    os.makedirs('%scoco/images/val2017/'%saved_coco_path)
if not os.path.exists('%scoco/images/test2017/'%saved_coco_path):
    os.makedirs('%scoco/images/test2017/'%saved_coco_path)
# 把训练集转化为COCO的json格式
l2c_test = Csv2CoCo(image_dir=image_dir,total_annos=total_csv_annotations)
test_instance = l2c_test.to_coco(test_keys)
l2c_test.save_coco_json(test_instance, '%scoco/annotations/instances_test2017.json'%saved_coco_path)
for file in test_keys:
    if not os.path.exists("%scoco/images/test2017/%s"%(saved_coco_path,file.split(os.sep)[-2]+'_'+file.split(os.sep)[-1])):
        print("%scoco/images/test2017/%s"%(saved_coco_path,file.split(os.sep)[-2]+'_'+file.split(os.sep)[-1]))
        shutil.copy(image_dir+file,"%scoco/images/test2017/%s"%(saved_coco_path,file.split(os.sep)[-2]+'_'+file.split(os.sep)[-1]))

test_n: 50080


In [14]:
with open('coco/annotations/instances_val2017.json', 'r') as f:
    data = json.load(f)

In [15]:
data['categories'] = data['categories'][:-1]

In [16]:
with open('coco/annotations/instances_val2017.json', 'w') as f:
    json.dump(data, f)